# PreProcessing

name: Bow 

Shoplocation: hot suite

discount: object -> float

False True: 0 1

rating_count

z-score


In [38]:
import re
import pandas as pd
import numpy as np

from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
# from sklearn.cross_validation import train_test_split

# Mở file và chỉnh sửa thông số hiển thị

In [ ]:
# open

items_df = pd.read_csv('ShoppeItem_thoiTrangNam_aoSoMi_0-949.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [65]:
items_df

,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,show_official_shop_label_in_title,rating_star,five_star,four_star,three_star,two_star,one_star,flash_sale,upcoming_flash_sale,price_min,price_min_before_discoun,price_max,price_max_before_discount,price,price_before_discount,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,historical_sold
0,"Áo sơ mi nam tay dài trơn đủ màu có size big LADOS-523, vải lụa mềm mát",TP. Hồ Chí Minh,1590652993,1609949438,19296445,False,45%,False,49,True,4.888889,771,48,12,3,3,None,None,9900000000,17900000000,9900000000,17900000000,9900000000,17900000000,None,435,32314,837,False,True,9,1,2445
1,Áo sơ mi nam trơn không nhăn LADOS-523,TP. Hồ Chí Minh,1507346805,1609949438,102602633,False,61%,False,50,True,4.847889,3736,238,92,20,37,None,None,6900000000,17900000000,6900000000,17900000000,6900000000,17900000000,None,1411,34074,4122,False,True,9,1,13361
2,"Áo sơ mi nam trơn đủ màu cao cấp tay dài LADOS-4523,sơ mi không nhăn",TP. Hồ Chí Minh,1564026320,1609949438,45923118,False,50%,False,45,True,4.861345,217,13,5,2,1,None,None,9900000000,19800000000,9900000000,19800000000,9900000000,19800000000,None,104,5119,238,False,True,9,0,720
3,"Áo sơ mi Nam họa tiết sọc trắng đen Boo Boo chất lụa Twill chéo mềm mại, thoải mái - FEAER DENIM",TP. Hồ Chí Minh,1594349418,1609949438,15600020,False,33%,False,4,True,4.957063,2257,57,7,3,6,None,None,17500000000,26000000000,17500000000,26000000000,17500000000,26000000000,None,3460,113754,2329,False,True,9,1,5060
4,Áo sơ mi nam cao cấp vải lụa THỜI TRANG THIẾT KẾ,TP. Hồ Chí Minh,1556543538,1609949438,53405900,True,28%,False,48,False,4.857749,430,25,11,0,5,None,None,25200000000,35000000000,25200000000,35000000000,25200000000,35000000000,None,5290,12595,497,False,True,4,0,1085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,[Ảnh thật ở sau] Áo sơ mi nam phối túi sọc thời trang nam đồ nam chất lượng,TP. Hồ Chí Minh,1587829639,1609949438,22119799,False,None,False,8,False,0.000000,0,0,0,0,0,None,None,19900000000,-1,19900000000,-1,19900000000,0,None,4,27,0,False,True,6,0,2
946,{HOT} ÁOSơ Mi Nam Chất Cotton Dài Tay AH122 [ thời trang nam],Hà Nội,1566640974,1609949438,43308464,False,None,False,5,False,5.000000,3,0,0,0,0,None,None,15900000000,-1,15900000000,-1,15900000000,0,None,366,1,3,False,False,3,0,10
947,SƠ MI THỜI TRANG NAM HỌA TIẾT NHIỀU Ô VUÔNG - KHX024,TP. Hồ Chí Minh,1531205353,1609949438,78744085,False,29%,False,6,False,4.500000,8,1,0,0,1,None,None,17000000000,23900000000,17000000000,23900000000,17000000000,23900000000,None,13,3,10,False,False,2,0,29
948,"Áo Sơ Mi Nam trắng dài tay Hàn Quốc form body, vải lụa Thái chống nhăn [Thời Trang Nam]",TP. Hồ Chí Minh,1563797429,1609949438,46152009,False,41%,False,6,False,4.500000,10,0,1,0,1,None,None,8201000000,13900000000,8201000000,13900000000,8201000000,13900000000,None,6606,53,12,False,True,2,0,30


Do dữ liệu trả về từ api của shopee những dữ liệu thiếu có dạng string: 'None' nên pandas sẽ không đếm được lượng dữ liệu thiếu này. Ta tiến hành thay 'None' thành pd.NA để có thể đếm dữ liệu thiếu chính xác.

In [68]:
# cond = items_df != 'None'
# items_df.where(cond, pd.NA)
items_df = items_df.replace('None', pd.NA)

In [19]:
items_df.describe()

,item_public_time,item_be_got_time,sell_time,options,rating_star,five_star,four_star,three_star,two_star,one_star,price_min,price_min_before_discoun,price_max,price_max_before_discount,price,price_before_discount,liked_count,view_count,cmt_count,images,video_info_list,historical_sold
count,9.500000e+02,9.500000e+02,9.500000e+02,950.000000,950.000000,950.00000,950.000000,950.000000,950.000000,950.000000,9.500000e+02,9.500000e+02,9.500000e+02,9.500000e+02,9.500000e+02,9.500000e+02,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000
mean,1.590147e+09,1.609949e+09,1.980261e+07,17.995789,1.043778,16.74000,0.886316,0.335789,0.094737,0.188421,2.709091e+10,2.166468e+10,2.781708e+10,2.215292e+10,2.709091e+10,2.166468e+10,270.623158,413.172632,18.358947,7.277895,0.083158,110.597895
std,1.939367e+07,0.000000e+00,1.939367e+07,14.578734,1.978081,164.46327,9.131155,3.341769,0.813139,1.453629,2.131242e+10,2.840180e+10,2.172379e+10,2.887830e+10,2.131242e+10,2.840180e+10,1195.808746,4244.789531,178.464764,2.134230,0.276266,838.882609
min,1.494652e+09,1.609949e+09,1.765580e+05,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.900000e+09,-1.000000e+00,3.900000e+09,-1.000000e+00,3.900000e+09,0.000000e+00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.584393e+09,1.609949e+09,5.599614e+06,8.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.509302e+10,-1.000000e+00,1.543250e+10,-1.000000e+00,1.509302e+10,0.000000e+00,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000
50%,1.596594e+09,1.609949e+09,1.335513e+07,12.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.133000e+10,1.700000e+10,2.185000e+10,1.771500e+10,2.133000e+10,1.700000e+10,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000
75%,1.604350e+09,1.609949e+09,2.555662e+07,24.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,3.159250e+10,3.179355e+10,3.296000e+10,3.284705e+10,3.159250e+10,3.179355e+10,2.000000,10.750000,0.000000,9.000000,0.000000,1.000000
max,1.609773e+09,1.609949e+09,1.152978e+08,75.000000,5.000000,3736.00000,238.000000,92.000000,20.000000,37.000000,2.245000e+11,1.980000e+11,2.245000e+11,1.980000e+11,2.245000e+11,1.980000e+11,17151.000000,113754.000000,4122.000000,9.000000,1.000000,13469.000000


In [69]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               950 non-null    object 
 1   shop_location                      946 non-null    object 
 2   item_public_time                   950 non-null    int64  
 3   item_be_got_time                   950 non-null    int64  
 4   sell_time                          950 non-null    int64  
 5   shopee_verified                    950 non-null    bool   
 6   discount                           559 non-null    object 
 7   is_adult                           950 non-null    bool   
 8   options                            950 non-null    int64  
 9   show_official_shop_label_in_title  950 non-null    bool   
 10  rating_star                        950 non-null    float64
 11  five_star                          950 non-null    int64  

Ta thấy 3 thuộc tính flash_sale, upcoming_flash_sale, coin_earn_label đều bị thiếu dữ liệu (0 non-null) nên ta tiến hành xóa bỏ ba thuộc tính này khỏi dataset.

In [70]:
items_df = items_df.drop('flash_sale', axis = 1)
items_df = items_df.drop('upcoming_flash_sale', axis = 1)
items_df = items_df.drop('coin_earn_label', axis = 1)

In [71]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               950 non-null    object 
 1   shop_location                      946 non-null    object 
 2   item_public_time                   950 non-null    int64  
 3   item_be_got_time                   950 non-null    int64  
 4   sell_time                          950 non-null    int64  
 5   shopee_verified                    950 non-null    bool   
 6   discount                           559 non-null    object 
 7   is_adult                           950 non-null    bool   
 8   options                            950 non-null    int64  
 9   show_official_shop_label_in_title  950 non-null    bool   
 10  rating_star                        950 non-null    float64
 11  five_star                          950 non-null    int64  

Kiểm tra lại ta thấy thuộc tính discount gần như có một nửa giá trị bị thiếu. Kiểm tra đối chiếu với các item trên shopee, ta thấy thực tế những mẫu bị thiếu dữ liệu ở thuộc tính discount là do không có giảm giá (discount = 0). Ta tiến hành sửa bằng cách thay giá trị NA tại cột này thành 0.

In [74]:
items_df['discount'] = items_df['discount'].fillna(0)

In [75]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               950 non-null    object 
 1   shop_location                      946 non-null    object 
 2   item_public_time                   950 non-null    int64  
 3   item_be_got_time                   950 non-null    int64  
 4   sell_time                          950 non-null    int64  
 5   shopee_verified                    950 non-null    bool   
 6   discount                           950 non-null    object 
 7   is_adult                           950 non-null    bool   
 8   options                            950 non-null    int64  
 9   show_official_shop_label_in_title  950 non-null    bool   
 10  rating_star                        950 non-null    float64
 11  five_star                          950 non-null    int64  

Đến đây ta thấy chỉ còn cột shop_location có một vài mẫu bị thiếu dữ liệu. Do số lượng mẫu thiếu dữ liệu ít nên ta có thể xóa những mẫu này.

In [78]:
items_df = items_df.dropna(axis = 0)

In [79]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 946 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   name                               946 non-null    object 
 1   shop_location                      946 non-null    object 
 2   item_public_time                   946 non-null    int64  
 3   item_be_got_time                   946 non-null    int64  
 4   sell_time                          946 non-null    int64  
 5   shopee_verified                    946 non-null    bool   
 6   discount                           946 non-null    object 
 7   is_adult                           946 non-null    bool   
 8   options                            946 non-null    int64  
 9   show_official_shop_label_in_title  946 non-null    bool   
 10  rating_star                        946 non-null    float64
 11  five_star                          946 non-null    int64  

Đến đây ta đã không còn giá trị thiếu, tuy nhiên có thể có giá trị không hợp lệ. Thật ra giá trị của những mẫu này đều đang hoạt động bình thường trên giao diện shopee nên khả năng lỗi là rất thấp. Ta vẫn kiểm tra lại như sau:
1. Có giá trị âm ở các cột dạng numeric không
2. So sánh price_min và price_max, price_min_before_discount và price_max_before_discount
3. So sánh price và price_before_discount, price_min và price_min_before_discount, price_max và price_max_before_discount

In [7]:
items_df.columns.values

array(['name', 'shop_location', 'item_public_time', 'item_be_got_time',
       'sell_time', 'shopee_verified', 'discount', 'is_adult', 'options',
       'show_official_shop_label_in_title', 'rating_star', 'five_star',
       'four_star', 'three_star', 'two_star', 'one_star', 'flash_sale',
       'upcoming_flash_sale', 'price_min', 'price_min_before_discoun',
       'price_max', 'price_max_before_discount', 'price',
       'price_before_discount', 'coin_earn_label', 'liked_count',
       'view_count', 'cmt_count', 'is_preferred_plus_seller',
       'show_free_shipping', 'images', 'video_info_list',
       'historical_sold'], dtype=object)

In [14]:
items_df['shop_location'].unique()

array(['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài', 'Bà Rịa - Vũng Tàu',
       'Phú Thọ', 'Bắc Ninh', 'Quảng Ninh', 'Lâm Đồng', 'None',
       'Hưng Yên', 'Hải Dương', 'Thái Bình'], dtype=object)

In [20]:
shop_locations = ['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài', 'Bà Rịa - Vũng Tàu',
       'Phú Thọ', 'Bắc Ninh', 'Quảng Ninh', 'Lâm Đồng', 'None',
       'Hưng Yên', 'Hải Dương', 'Thái Bình']

In [24]:
X = items_df['shop_location']
X = X[:, np.newaxis]
X

array([['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Nước ngoài'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Hà Nội'],
       ['Bà Rịa - Vũng Tàu'],
       ['TP. Hồ Chí Minh'],
       ['Nước ngoài'],
       ['Nước ngoài'],
       ['TP. Hồ Chí Minh'],
       ['Hà Nội'],
       ['Nước ngoài'],

In [30]:
enc = OneHotEncoder(categories=[shop_locations])
enc.fit(X)

OneHotEncoder(categories=[['TP. Hồ Chí Minh', 'Hà Nội', 'Nước ngoài',
                           'Bà Rịa - Vũng Tàu', 'Phú Thọ', 'Bắc Ninh',
                           'Quảng Ninh', 'Lâm Đồng', 'None', 'Hưng Yên',
                           'Hải Dương', 'Thái Bình']])

In [37]:
X_ = enc.transform(X).toarray()
print(X[351])
print(X_[351])

['Nước ngoài']
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
items_df['name'].describe()

count                                    950
unique                                   648
top       Áo Khoác Sơ Mi Caro Thời Trang Nam
freq                                     140
Name: name, dtype: object

In [40]:
simple_imp= SimpleImputer()
imp_mode = SimpleImputer(strategy='most_frequent')

#shop_location
knn_imp = KNNImputer(n_neighbors = 3)

In [ ]:
imp_ss_pipe = make_pipeline(simple_imp, StandardScaler())

In [ ]:
steps = list()
steps.append(('scaler', MinMaxScaler()))
pipeline = Pipeline(steps=steps)

In [28]:
type(items_df['rating_count'][0])

str

In [22]:
num_duplicated_rows = items_df.index.duplicated(keep='first')
num_duplicated_rows = np.count_nonzero(num_duplicated_rows)
num_duplicated_rows

0

In [27]:
dtypes = items_df.dtypes
dtypes

name                                  object
shop_location                         object
shopee_verified                         bool
discount                              object
is_adult                                bool
options                                int64
show_official_shop_label_in_title       bool
rating_star                          float64
rating_count                          object
flash_sale                            object
upcoming_flash_sale                   object
price_min                              int64
price_min_before_discoun               int64
price_max                              int64
price_max_before_discount              int64
price                                  int64
price_before_discount                  int64
coin_earn_label                       object
liked_count                            int64
view_count                             int64
cmt_count                              int64
is_preferred_plus_seller                bool
show_free_